In [1]:
import numpy as np
import pandas as pd
import pickle
import math
import os
import matplotlib.pyplot as plt
###################
#disp & nf data
disp=[];nf=[]
for i in range(12000):
    df=pd.read_excel('data/raw_data/12000cases/CaseJuly'+str(i+1)+'.xlsx')
    nf.append(df.iloc[0,:7].values.astype('float64'))
    df=df.values[2:,list(range(3,3*7+1,3))]#df=df.values[1:,3]
    disp.append(df)

for i in range(121):
    df=pd.read_excel('data/raw_data/121cases_99_99/Case-08-20.'+str(i)+'.xlsx')
    nf.append(df.iloc[0,:7].values.astype('float64'))
    df=df.values[2:,list(range(3,3*7+1,3))]#df=df.values[1:,3]
    disp.append(df)
    
disp=np.stack(disp,axis=0)
nf=np.stack(nf,axis=0)
print(disp.shape,nf.shape)
disp=disp.astype('float64')
print(np.min(disp),np.max(disp),np.mean(disp),np.std(disp))
print(np.min(nf),np.max(nf),np.mean(nf),np.std(nf))
np.save('data/disp',disp)
np.save('data/nf',nf)

#area data
area=pd.read_excel('data/raw_data/12000cases/INPUT_12000.xlsx')
area=area.values[:,1:];print(area.shape)#(12000, 120)

area2=pd.read_excel('data/raw_data/121cases_99_99/Case/INPUT_120.xlsx')
area2=area2.values[:,1:];print(area2.shape)

area=np.concatenate((area,area2),axis=0)
np.save('data/data_cablearea',area)

(12000, 120)
(121, 120)


In [1]:
import numpy as np
import pandas as pd
import pickle
import math
import os
import matplotlib.pyplot as plt
###################
disp=[];nf=[]
f_names=['multidamage_1','multidamage_2','multidamage_3','multidamage_4']
n_files=[range(20000),range(20000,4000),range(40000,60000),range(60000,75000)]
for i,f in enumerate(f_names):
    print(f,len(disp))
    for j in n_files[i]:
        df=pd.read_excel('data/raw_data/'+f+'/Mul'+str(j+1)+'.xlsx')
        nf.append(df.iloc[0,:7].values.astype('float64'))
        df=df.values[2:,list(range(3,3*7+1,3))]#df=df.values[1:,3]
        disp.append(df)
    
disp=np.stack(disp,axis=0)
nf=np.stack(nf,axis=0)
print(disp.shape,nf.shape)
disp=disp.astype('float64')
print(np.min(disp),np.max(disp),np.mean(disp),np.std(disp))
print(np.min(nf),np.max(nf),np.mean(nf),np.std(nf))
np.save('data/disp_multi',disp)
np.save('data/nf_multi',nf)

area=pd.read_excel('data/raw_data/multidamage_1/CASES/INPUT.xlsx')
area=area.values[1:,1:];print(area.shape)#exclude no damage
np.save('data/data_cablearea_multi',area)

multidamage_1
multidamage_2
multidamage_3
multidamage_4
(55000, 365, 7) (55000, 7)
-1.0 1.0 -0.03725372321987086 0.3643339274707541
0.2825976374837506 1.590558445070064 1.0023826613664841 0.4263608059685357


In [1]:
import numpy as np
import pandas as pd
import pickle
import math
import matplotlib.pyplot as plt

node_loc=(pd.read_excel('data/raw_data/node_location.xlsx',header=None).values)
element_cable=(pd.read_excel('data/raw_data/element_cable.xlsx').iloc[:,[1,2,4,5]]).values
element_tower=(pd.read_excel('data/raw_data/element_tower.xlsx').iloc[:,[1,2,4,5]]).values
element_girder=(pd.read_excel('data/raw_data/element_girder.xlsx').iloc[:,[1,2,4,5]]).values
element_crossbeam=(pd.read_excel('data/raw_data/element_crossbeam.xlsx').iloc[:,[1,2,4,5]]).values
element_virtual=(pd.read_excel('data/raw_data/element_virtual.xlsx',header=None)).values
element_virtual=element_virtual-1
elements=np.concatenate((element_cable,element_tower,element_girder,element_crossbeam),axis=0)
elements[:,[0,1]]=elements[:,[0,1]]-1
print(np.min(elements[:,[0,1]]),np.max(elements[:,[0,1]]))
#adj_mx
num_node=len(node_loc);print('num node: ',num_node)
adj_mx = np.zeros((num_node, num_node), dtype=np.float32)
adj_mx[elements[:,0],elements[:,1]]=1
adj_mx[element_virtual[:,0],element_virtual[:,1]]=1
#adj_mx = np.maximum(adj_mx, adj_mx.transpose())#symmetric
print('num edge: ',np.sum(adj_mx))

#dist_mx
dist_mx = np.zeros((num_node, num_node), dtype=np.float32)
dist_mx[:] = np.inf
for i in range(num_node):
    for j in range(num_node):
        x = float(node_loc[i][0]) - float(node_loc[j][0])
        y = float(node_loc[i][1]) - float(node_loc[j][1])
        z = float(node_loc[i][2]) - float(node_loc[j][2])
        x = math.pow(x,2)
        y = math.pow(y,2)
        z = math.pow(z,2)
        dis = math.sqrt(x+y+z)
        dist_mx[i, j] = dis
dist_mx=adj_mx*dist_mx
dist_mx = np.maximum(dist_mx, dist_mx.transpose())#symmetric
#virtual node
virtual_mx=np.zeros((num_node, num_node), dtype=np.float32)
virtual_mx[element_virtual[:,0],element_virtual[:,1]]=1
virtual_mx = np.maximum(virtual_mx, virtual_mx.transpose())#symmetric

type_c_mx=np.zeros((num_node, num_node), dtype=np.float32)
type_t_mx=type_c_mx.copy();type_g_mx=type_c_mx.copy();type_cb_mx=type_c_mx.copy()
type_c_mx[element_cable[:,0]-1,element_cable[:,1]-1]=1
type_t_mx[element_tower[:,0]-1,element_tower[:,1]-1]=1
type_g_mx[element_girder[:,0]-1,element_girder[:,1]-1]=1
type_cb_mx[element_crossbeam[:,0]-1,element_crossbeam[:,1]-1]=1
print(np.sum(type_c_mx),np.sum(type_t_mx),np.sum(type_g_mx),np.sum(type_cb_mx))

np.save('data/sensor_graph/adj_mx',adj_mx)
np.save('data/sensor_graph/dist_mx',dist_mx)
np.save('data/sensor_graph/virtual_mx',virtual_mx)
np.save('data/sensor_graph/type_c_mx',type_c_mx)
np.save('data/sensor_graph/type_t_mx',type_t_mx)
np.save('data/sensor_graph/type_g_mx',type_g_mx)
np.save('data/sensor_graph/type_cb_mx',type_cb_mx)

0 358
num node:  365
num edge:  486.0
120.0 48.0 192.0 120.0
